In [ ]:
import torch
from model.modeling_UniBioseq import UniBioseqForEmbedding
from model.tokenization_UniBioseq import UBSLMTokenizer
model_file = "./load_files/BiooBang_FM"

/opt/user/hsj/anaconda3/envs/UniBioseq_torch211/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-08-01 14:51:07,778] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [ ]:
#init UniBioseqForEmbedding or UniBioseqForstructureEmbedding_convbert_MRL
model_fm = UniBioseqForEmbedding.from_pretrained(model_file)
tokenizer = UBSLMTokenizer.from_pretrained(model_file)
input_seq = "MTNSQALIKTLDFQLDIQSDNESLLYDATLEARSVYNQSIRLAKEGVDWNVIPDRVAEDASLVKNTIQRVVAKALGAMENYYEYDDYNKPSHTKDGTYPLRTNYEEGYNLSLTDDGDVAFRISAKPYKHVKGVLDGDDAHLDILKTALESDAWTIGTAEALFRNGNAELHVNVTSSEGTVRDKENSRTVVGVDVNEDNVALTALSKGSVEDTVVIEFPDIKFKRHRYFTMRKRVQNSGKDSIHDTLEGREERFVRDRLHKVSRHIVEWSRQFEKPCIVFEDLKEMRDGLDYGTRMNRRLHRLPFRALQHYTSYKAAFAGIPTAWINPEYTSQCCPMCGHTERSNRHKKRFKCQSCSHQDHADRSASVNIAVKGVEEYQEWTVPALNSLPVVRKVRRQASGAVDAPTVTHDAVRGYQTDGVAGVSD"
input_ids = torch.tensor([tokenizer.encode(input_seq)])
#run model
model_fm = model_fm.to("cuda")
result_fm = model_fm(input_ids.to(model_fm.device))['hidden_states'][0][1:-1] #UniBioseqForEmbedding

No vocab_list supplied for folding model, assuming the UBL vocabulary!


In [ ]:
#print if result shape == input length
print(result_fm.shape[0]==len(input_seq))

True


CDS conditional generation

In [ ]:
from model.tokenization_UniBioseq import UBSLMTokenizer
from model.modeling_UniBioseq import UniBioseqForCausalLM
from model.UBL_utils import new_logits_processor, forbid_aa
import time
from transformers.generation.logits_process import LogitsProcessorList
import torch
from Bio.Seq import Seq
load_file = "./load_files/BiooBang_decoder"
logits_processor = LogitsProcessorList()
logits_processor.append(new_logits_processor(forbid_aa()))

tokenizer = UBSLMTokenizer.from_pretrained(load_file)

print("load model!")
model = UniBioseqForCausalLM.from_pretrained(load_file, device_map='auto')

input_gfp = "MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITLGMDELYK"
input_mcherry = "MVSKGEEDNMAIIKEFMRFKVHMEGSVNGHEFEIEGEGEGRPYEGTQTAKLKVTKGGPLPFAWDILSPQFMYGSKAYVKHPADIPDYLKLSFPEGFKWERVMNFEDGGVVTVTQDSSLQDGEFIYKVKLRGTNFPSDGPVMQKKTMGWEASSERMYPEDGALKGEIKQRLKLKDGGHYDAEVKTTYKAKKPVQLPGAYNVNIKLDITSHNEDYTIVEQYERAEGRHSTGGMDELYKS"

input = input_gfp
input_ids = torch.tensor([tokenizer.encode(input)+[36]]).to(model.device)
num_beams = 50
max_length = 4*len(input)+6

print("start generation!")
start_time = time.time()
result = model.generate(input_ids, max_length = max_length, num_beams = num_beams, logits_processor=logits_processor, low_memory=True, num_return_sequences=num_beams)
end_time = time.time()
run_time = end_time - start_time
print(f"finish!!!\Run Time: {run_time} 秒")

print("mRNA results:")
result_tok_list = []
for i in range(result.shape[0]):
    result_tok = tokenizer.decode(result[i][len(input)+3:].tolist()).replace(" ","").upper()
    
    aa_generate = Seq(result_tok).translate()
    if aa_generate[:-1] != input:
        print("FALSE: {}".format(result_tok))
    else:
        result_tok_list.append(result_tok)
        print(result_tok)